# Modelos de regresión

#### Importación de bibliotecas

In [1]:
%load_ext autoreload
%autoreload 1

import sys
import os

project_root = os.path.abspath("..")
src_path = os.path.join(project_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)


import pandas as pd
import numpy as np
import unidecode
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import (r2_score, mean_absolute_error,
    mean_absolute_percentage_error, roc_auc_score)
from sklearn.pipeline import Pipeline

from data_analysis_octopus import (DataViz, detect_outliers_iqr, transform_outliers,
    get_varclushi, get_pca, get_kbest, count_percentage, create_feature_dataframe)


def freq_discrete(df, features):
    for feature in features:
        print(f"Feature: {feature}")
        abs_ = df[feature].value_counts(dropna=False).to_frame().rename(columns={"count": "Absolute frequency"})
        rel_ = df[feature].value_counts(dropna=False, normalize= True).to_frame().rename(columns={"proportion": "Relative frequency"})
        freq = abs_.join(rel_)
        freq["Accumulated frequency"] = freq["Absolute frequency"].cumsum()
        freq["Accumulated %"] = freq["Relative frequency"].cumsum()
        freq["Absolute frequency"] = freq["Absolute frequency"].map(lambda x: "{:,.0f}".format(x))
        freq["Relative frequency"] = freq["Relative frequency"].map(lambda x: "{:,.2%}".format(x))
        freq["Accumulated frequency"] = freq["Accumulated frequency"].map(lambda x: "{:,.0f}".format(x))
        freq["Accumulated %"] = freq["Accumulated %"].map(lambda x: "{:,.2%}".format(x))
        display(freq)


def clean_name_columns(columns):
    cleaned_columns_dict = {}
    for col in columns:
        
        processed_col = unidecode.unidecode(
            col.lower()
               .replace("¿", "")
               .replace("?", "")
               .replace(" (numérica)", "")
               .replace(" (numérico)", "")
               .replace(" ", "_")
        )

        cleaned_columns_dict[col] = processed_col

    return cleaned_columns_dict


def classify_colors(color):

    if color != "azul":
        return "no es azul"
    
    return color


def classify_pets(pet):

    if pet in ["perrro", "perro", "perros"]:
        return "perro"
    
    if pet in ["gato"]:
        return "gato"
    
    return "otro"


def classify_tattoos(n_tattoos):
    if n_tattoos <= 0:
        return "sin tatuajes"
    
    if n_tattoos > 0:
        return "con tatuajes"
    
    return float('nan')


def clean_text(text):
    text = unidecode.unidecode(text.lower())
    return text

## Procesamiento de la información

### Lectura de los datos

In [2]:
filename = "/workspace/data/Team-calor- o-team-frío.csv"
raw_df = pd.read_csv(filename).drop(columns="Marca temporal")

new_name_columns = clean_name_columns(raw_df.columns)
raw_df = raw_df.rename(columns=new_name_columns)
raw_df.head()

,color_primario_favorito,edad_en_anos,estatura_en_metros,tipo_de_personalidad,numero_de_vasos_de_agua_que_tomas_al_dia,dia_o_noche,actividad_fisica,mascota_favorita,chile_del_que_pica_o_del_que_no_pica,numero_de_hermanos,las_quesadillas_van_con_queso,numero_de_tatuajes,team_frio_o_team_calor
0,Azul,29,1.92,Introvertido,6,Noche,¿Qué es eso?,Perrro,Del que pica,2,Con queso,0,NaN
1,Azul,26,1.57,Introvertido,5,Noche,¿Qué es eso?,Perrro,Del que pica,0,Con queso,0,NaN
2,Azul,25,1.71,Introvertido,3,Noche,Si,Perrro,Del que pica,1,Con queso,0,NaN
3,Rojo,27,1.78,Introvertido,4,Noche,¿Qué es eso?,Perrro,Del que pica,1,Sin queso,0,NaN
4,Azul,23,1.7,Introvertido,4,Noche,Si,Gato,Del que no pica,2,Con queso,0,NaN


### Preprocesamiento

In [3]:
categoricas = [
    "color_primario_favorito",
    "tipo_de_personalidad",
    "dia_o_noche",
    "actividad_fisica",
    "mascota_favorita",
    "chile_del_que_pica_o_del_que_no_pica",
    "las_quesadillas_van_con_queso",
    "team_frio_o_team_calor"
]

numericas = [
    "edad_en_anos",
    "numero_de_hermanos",
    "estatura_en_metros",
    "numero_de_vasos_de_agua_que_tomas_al_dia",
    "numero_de_tatuajes"
]

In [4]:
processed_df = raw_df.copy()

processed_df[categoricas] = processed_df[categoricas].fillna("")
processed_df = processed_df.map(lambda value: value.strip())
processed_df[categoricas] = processed_df[categoricas].map(clean_text)

# Reclasificaicón de categorías
processed_df["color_primario_favorito"] = processed_df["color_primario_favorito"].apply(classify_colors)
processed_df["mascota_favorita"] = processed_df["mascota_favorita"].apply(classify_pets)
# Se transforman a con tatuajes o sin tatuajes, ya que, al detectar outliers, 
# mediante IQR, aquellos con valores mayoes a 0, son valores atípicos, 
# para conservar la información se recategoriza en: 'con tatuajes' & 'sin tatuajes
processed_df["tatuajes"] = processed_df["numero_de_tatuajes"].apply(pd.to_numeric, errors='coerce').apply(classify_tattoos)

In [5]:
categoricas.append("tatuajes")

In [6]:
freq_discrete(processed_df, categoricas)

Feature: color_primario_favorito


,Absolute frequency,Relative frequency,Accumulated frequency,Accumulated %
color_primario_favorito,,,,
azul,69,57.98%,69,57.98%
no es azul,50,42.02%,119,100.00%


Feature: tipo_de_personalidad


,Absolute frequency,Relative frequency,Accumulated frequency,Accumulated %
tipo_de_personalidad,,,,
introvertido,74,62.18%,74,62.18%
extrovertido,45,37.82%,119,100.00%


Feature: dia_o_noche


...

,Absolute frequency,Relative frequency,Accumulated frequency,Accumulated %
las_quesadillas_van_con_queso,,,,
con queso,90,75.63%,90,75.63%
sin queso,29,24.37%,119,100.00%


Feature: team_frio_o_team_calor


,Absolute frequency,Relative frequency,Accumulated frequency,Accumulated %
team_frio_o_team_calor,,,,
team frio,87,73.11%,87,73.11%
team calor,18,15.13%,105,88.24%
,14,11.76%,119,100.00%


Feature: tatuajes


,Absolute frequency,Relative frequency,Accumulated frequency,Accumulated %
tatuajes,,,,
sin tatuajes,93,78.15%,93,78.15%
con tatuajes,25,21.01%,118,99.16%
NaN,1,0.84%,119,100.00%


In [7]:
import cufflinks as cf
cf.go_offline()

for var in numericas:
    display(processed_df[var].iplot(kind="hist", theme="solar", title=var))

None

None

None

None

None

In [8]:
processed_df[numericas] = processed_df[numericas].apply(pd.to_numeric, errors='coerce')

#### Detección y remoción de variables poco pobladas

In [9]:
# No es necesario eliminar variables poco pobladas
THRESHOLD = 65
completitud_df = DataViz.completitud(processed_df)
completitud_df[completitud_df["% valores nulos"] >= THRESHOLD]

,Total de nulos,% valores nulos


#### Detección y remoción de valores extremos

In [10]:
for c in numericas:
    lower_bound, upper_bound = detect_outliers_iqr(processed_df, c)
    processed_df = transform_outliers(processed_df, c, lower_bound, upper_bound)

#### Detección y tratamiento de valores ausentes

In [11]:
for col in numericas:
    media = processed_df[col].median()
    processed_df.loc[:, col] = processed_df[col].replace("", np.nan).fillna(media)  

for col in categoricas:
    moda = processed_df[col].mode()[0]
    processed_df.loc[:, col] = processed_df[col].replace("", np.nan).fillna(moda)

In [12]:
processed_df[numericas].describe()

,edad_en_anos,numero_de_hermanos,estatura_en_metros,numero_de_vasos_de_agua_que_tomas_al_dia,numero_de_tatuajes
count,119.000000,119.000000,119.000000,119.000000,119.0
mean,24.756303,1.634454,1.749118,6.327731,0.0
std,4.085871,0.908289,0.149020,2.690565,0.0
min,14.500000,-0.500000,1.400000,-2.000000,0.0
25%,22.000000,1.000000,1.640000,4.000000,0.0
50%,24.000000,2.000000,1.715000,6.000000,0.0
75%,27.000000,2.000000,1.800000,8.000000,0.0
max,34.500000,3.500000,2.040000,14.000000,0.0


#### Análisis de correlación

In [13]:
processed_df[numericas].corr()

,edad_en_anos,numero_de_hermanos,estatura_en_metros,numero_de_vasos_de_agua_que_tomas_al_dia,numero_de_tatuajes
edad_en_anos,1.000000,0.137353,0.108485,0.001545,NaN
numero_de_hermanos,0.137353,1.000000,0.171968,0.077180,NaN
estatura_en_metros,0.108485,0.171968,1.000000,0.157770,NaN
numero_de_vasos_de_agua_que_tomas_al_dia,0.001545,0.077180,0.157770,1.000000,NaN
numero_de_tatuajes,NaN,NaN,NaN,NaN,NaN


#### Detección y remoción de variables unitarias (unarias)
cuando el 90% de la información se agrupe dentro una sola variable

In [14]:
dfs_list = [ ]
for c in processed_df.columns:
    tmp_count = count_percentage(processed_df, c)
    
    tmp_df = tmp_count[tmp_count["porcentaje"] >= 90]

    if not tmp_df.empty:
        dfs_list.append(create_feature_dataframe(tmp_df, c))

unit_vars = pd.concat(dfs_list).reset_index(drop=True)
unit_vars

,feature,category,conteo,porcentaje
0,numero_de_tatuajes,0.0,119,100.0


In [15]:
processed_df = processed_df.drop(columns=unit_vars["feature"].to_list())

#### Tratamiento de variables categoricas

In [16]:
dummies_df = pd.get_dummies(processed_df[categoricas], drop_first=True, dtype=int)
processed_df = pd.concat([processed_df, dummies_df], axis=1)

#### Escalamiento de los datos

In [17]:
mask_columns = processed_df.columns.isin(categoricas) == False
subset_df = processed_df.loc[:, mask_columns]

minmax = MinMaxScaler()
minmax_df = pd.DataFrame(
    minmax.fit_transform(subset_df),
    columns=subset_df.columns
)

X = minmax_df.loc[:, minmax_df.columns != "estatura_en_metros"]
y = minmax_df[["estatura_en_metros"]]


## Selección de variables

#### Clustering de variables

In [18]:
vc_df, vc_rsquare_df, best_features = get_varclushi(X)
best_features_df = best_features.to_frame().reset_index(drop=True)
best_features_df

,Variable
0,actividad_fisica_si
1,mascota_favorita_otro
2,actividad_fisica_no
3,chile_del_que_pica_o_del_que_no_pica_del que pica
4,team_frio_o_team_calor_team frio
5,numero_de_hermanos


In [19]:
top_features_vc = best_features_df.head(5)["Variable"].to_list()

### Kbest

In [20]:
scores_df = get_kbest(X, y, k=10)
scores_df

,Feature,Score
10,chile_del_que_pica_o_del_que_no_pica_del que pica,4.499563
2,numero_de_hermanos,3.565464
3,color_primario_favorito_no es azul,3.240512
11,las_quesadillas_van_con_queso_sin queso,3.192708
12,team_frio_o_team_calor_team frio,3.022826
1,numero_de_vasos_de_agua_que_tomas_al_dia,2.986635
7,actividad_fisica_si,2.726070
0,edad_en_anos,1.393381
5,dia_o_noche_noche,1.296043
4,tipo_de_personalidad_introvertido,0.879603


In [21]:
top_features_kbest = scores_df.head(5)["Feature"].to_list()

## Modelado

### Regresión Lineal

##### Selección de mejores features

In [22]:
best_features = list(set(top_features_vc + top_features_kbest))
X = processed_df[best_features]
y = processed_df[["estatura_en_metros"]]

X.head()

,mascota_favorita_otro,actividad_fisica_si,actividad_fisica_no,chile_del_que_pica_o_del_que_no_pica_del que pica,color_primario_favorito_no es azul,las_quesadillas_van_con_queso_sin queso,team_frio_o_team_calor_team frio,numero_de_hermanos
0,0,0,0,1,0,0,1,2.0
1,0,0,0,1,0,0,1,0.0
2,0,1,0,1,0,0,1,1.0
3,0,0,0,1,1,1,1,1.0
4,0,1,0,0,0,0,1,2.0


#### Train-test split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [24]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

scaler_X.fit(X_train)
scaler_y.fit(y_train.values.reshape(-1, 1))

# Transformar los datos de entrenamiento y prueba
scaled_X_train = scaler_X.transform(X_train)
scaled_X_test = scaler_X.transform(X_test)
scaled_y_train = scaler_y.transform(y_train.values.reshape(-1, 1))
scaled_y_test = scaler_y.transform(y_test.values.reshape(-1, 1))

In [25]:
linreg = LinearRegression()
linreg.fit(scaled_X_train, scaled_y_train)
linreg.score(scaled_X_train, scaled_y_train)

0.16515194768072028

#### Cross-validation

In [26]:
ls_res = cross_val_score(X=scaled_X_train, y=scaled_y_train, estimator=linreg, cv=4, scoring="r2", n_jobs=-1)
ls_res

array([-0.16650925,  0.03585047,  0.12567758, -0.19194495])

In [27]:
ls_res.mean(), ls_res.std()

(-0.04923154087043735, 0.1341205878446726)

#### Validación en test

In [28]:
linreg.score(scaled_X_test, scaled_y_test)

0.11759285298314381

#### Predicción de test

In [29]:
scaled_y_pred = linreg.predict(scaled_X_test)

In [30]:
pred_df = pd.DataFrame(scaler_y.inverse_transform(scaled_y_pred), columns=["y_pred"])
pred_df.head()

,y_pred
0,1.787715
1,1.787715
2,1.768727
3,1.827918
4,1.706418


#### Métricas de performance

##### R²

In [31]:
r2_score(y_true=scaled_y_test, y_pred=scaled_y_pred)

0.11759285298314381

##### PEMA

In [32]:
pema = mean_absolute_percentage_error(y_true=scaled_y_test, y_pred=scaled_y_pred)
f"{pema:,.2%}"

'37.45%'

##### MAE

In [33]:
mean_absolute_error(y_true=scaled_y_test, y_pred=scaled_y_pred)

0.1753621788283981

##### Interpretación

In [34]:
linreg.intercept_

array([0.23118071])

In [35]:
df_res = pd.DataFrame(data=zip(X_train.columns, linreg.coef_[0]), columns=["feature", "weight"])
df_res["weight"] = df_res["weight"].round(8)

df_res.sort_values(by="weight", ascending=False)

,feature,weight
7,numero_de_hermanos,0.167512
6,team_frio_o_team_calor_team frio,0.127028
5,las_quesadillas_van_con_queso_sin queso,0.100242
1,actividad_fisica_si,0.100150
0,mascota_favorita_otro,0.058481
3,chile_del_que_pica_o_del_que_no_pica_del que pica,0.042752
2,actividad_fisica_no,-0.000000
4,color_primario_favorito_no es azul,-0.029670


### Regresión Logística

In [36]:
label_encoder = LabelEncoder()
processed_df['color_primario_favorito_encoded'] = label_encoder.fit_transform(processed_df['color_primario_favorito'])

In [37]:
subset = [
    "edad_en_anos",
    "estatura_en_metros",
    "numero_de_vasos_de_agua_que_tomas_al_dia",
    "numero_de_hermanos",
    "tipo_de_personalidad_introvertido",
    "dia_o_noche_noche",
    "actividad_fisica_no",
    "actividad_fisica_si",
    "mascota_favorita_otro",
    "mascota_favorita_perro",
    "chile_del_que_pica_o_del_que_no_pica_del que pica",
    "las_quesadillas_van_con_queso_sin queso",
    "team_frio_o_team_calor_team frio",
    "tatuajes_sin tatuajes"
]

X = processed_df[subset]
y = processed_df["color_primario_favorito_encoded"]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
pipe = Pipeline([("modelo", LogisticRegression(max_iter=1_000))])
pipe.fit(X_train, y_train)

Pipeline(steps=[('modelo', LogisticRegression(max_iter=1000))])

In [40]:
ls_scores = cross_val_score(X=X_train, y=y_train, cv=4, n_jobs=-1, estimator=pipe, scoring="roc_auc")
ls_scores.mean(), ls_scores.std()

(0.5149038461538461, 0.07475920982811246)

In [41]:
pipe.score(X_test, y_test)

0.5666666666666667

#### Métricas de performance

##### ROC-AUC

In [42]:
roc_auc_score(y_score=pipe.predict(X_test), y_true=y_test)

0.5277777777777778

In [45]:
data = zip(X_train.columns, pipe[-1].coef_[0])
coef_df = pd.DataFrame(data, columns=["feature", "coef"])
coef_df["abs_coef"] = coef_df["coef"].abs()

coef_df = (coef_df.sort_values(by="abs_coef", ascending=False)
    .reset_index(drop=True)
)

coef_df

,feature,coef,abs_coef
0,mascota_favorita_otro,1.280543,1.280543
1,team_frio_o_team_calor_team frio,-0.892993,0.892993
2,numero_de_hermanos,-0.603168,0.603168
3,dia_o_noche_noche,-0.535778,0.535778
4,mascota_favorita_perro,-0.388888,0.388888
5,chile_del_que_pica_o_del_que_no_pica_del que pica,0.352941,0.352941
6,actividad_fisica_no,0.301009,0.301009
7,tatuajes_sin tatuajes,-0.285675,0.285675
8,estatura_en_metros,-0.250006,0.250006
9,las_quesadillas_van_con_queso_sin queso,0.120631,0.120631


## Conclusiones

**Regresión Lineal**

En cuanto al desempeño, tanto R² (11%), MAE (0.17) noS dicen que el modelo explican muy poca variabilidad de la estatura y hay poca precisión en las predicciones. En cuanto al error absoluto medio porcentual (MAPE) es del 37.45%, lo que indica una diferncia significativa entre las predicciones y los valores reales de la estatura. <br>

En cuanto a las características, el número de hermanos y team (frío o calor), son las caracetrisitcas con mayor influencia, con coeficientes más altos, en comparación con otros. Tener más hermanos y ser team frío están asociados a una estatura ligereamente mayor. <br>

El modelo de regresión lineal tiene un desempeño limitado para inferir la estatura. Me parece que en se debe, en parte, a que las características utilizadas no tienen un impacto significativo en la estatura, o no son causales de ser más pequeños o altos.

**Regresión Logística**

El score de ROC-AUC es de 0.48, con lo que tiene un pobre rendimiento para inferir si el color favorita de una persona es azul o no lo es. <br>

En cuanto a las características, lo que hace más probable que una persona tenga por color favorito el azul son: preferencia por mascotas distintas a las comunes, no realizar actividades físicas y no tener tatuajes.  <br>

Y aquellas características que hace menos probable que una persona elija el azul son: que sea team frío,  a mayor cantidad de hermanos, una estatura más baja. 


